# Final Project - Media / Twitter Analysis

The first step is to get to know what the Media is speaking about, in order to do this, I will extract the information from the main Digital Newspapers in Argentina.

In [ ]:
# -*- coding: utf-8 -*-
import bs4 as bs
from urllib import urlopen as uReq
import sys
import numpy as np
import re
import string
import pandas as pd
import pylab as pl
import matplotlib as plt
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

# La Nación

In [ ]:
# I request the URL, read, parsed with BS and saved it in the variable soup

# I create the data_ranges and set the dates for which I will scrap the data
date_range = pd.date_range('2017-04-28',periods=8, freq='d')
date_range = date_range.strftime("%d/%m/%Y")


# creates the file, using dd/mm/yyyy in the filename
filename = '/users/hnesman/Downloads/DATA_Science/La Nacion.txt'
    # opens the file
f = open(filename,'w')
    # creates and writes the headers of the file
headers = "Date|Sección|Title|Encabezado\n"
f.write(headers)

# For each day, I have to iterate over Section, so I create the list of sections, I use the url code for each section
secciones = ['c30','c272','c7','c28','c131','c337','c432','c7775']

# I will use this map later for labeling each article
# sec_dic = {'c30':'política','c272':'economía','c7':'elmundo','c28':'opinión','c131':'deportiva','c337':'elcampo',
#           'c432':'tecnología','c7775':'seguridad'}

# Loop over each section within each day
for i in date_range:
        for sec in secciones:
        # creates de url link with the date and the section
            url = 'http://servicios.lanacion.com.ar/archivo-f%s-%s' % (i,sec)
            # creates de BS object from the URL
            sauce = uReq(url)
            # does the parsing
            soup = bs.BeautifulSoup(sauce,'html.parser')
            
            # gets the tabs containing the date from the BS object, in this case I take the date from the url
            date = soup.findAll('div',{'class':'titFecha'})
            # converts date into string 
            date = str(date)
            # this is to extract the date from the date container
            fecha = '"titFecha">'
            date = date[date.find(fecha)+len(fecha):date.find(fecha)+21]
            # slices date to get dd/mm/yyyy
            day = date[:2]
            month = date[3:5]
            year = date[6:12]
            date = (day+'/'+month+'/'+year)
            print date + ' - ' + sec

            
            # creates the list of containers of the data I want from the 'url'
            acumulados = soup.findAll('li',{'class':'acumulados'})
            
            #creates the loop for each element of the list acumulados
            for acum in acumulados:
                # gets the title (which is the url link)
                title =  acum.a['href']
                print title
                # gets the body of the element
                encabezado = acum.p
                print encabezado,'\n'
                #writes the info (date, section, title & encabezado) into the file
                f.write(str(date) + '|'+ str(sec) + '|' + str(title) + '|' + str(encabezado).replace('<p>','').replace('</p>','') + '\n')

f.close()

In [ ]:
# Open the file and put into a dataFrame
LaNacion = pd.read_table('/users/hnesman/Downloads/DATA_Science/La Nacion.txt',delimiter="|")

LaNacion.columns = ['Date', 'SecCode','Title','Encabezado']

# Creates the Dictionary that I will use to map the Section Codes from the 'url's
sec_dic = {'c30':'política','c272':'economía','c7':'elmundo','c28':'opinión','c131':'deportiva','c337':'elcampo',
          'c432':'tecnología','c7775':'seguridad'}
# Do the mapping
LaNacion['Sec Label'] = LaNacion.SecCode.apply(lambda x: sec_dic[str(x)])

# Add 'Media' column, which is the label for the name of the Newspaper
LaNacion['Media'] = 'La Nacion'

# Drop the column 'SecCode'
LaNacion = LaNacion.drop('SecCode',axis=1)

# Set the names of the columns
LaNacion.columns = ['Date','Title','Encabezado','SecLabel','Media']

# Save to .csv file
LaNacion.to_csv('/users/hnesman/Downloads/DATA_Science/LaNacion.csv')
LaNacion.head()

# La Voz

In [ ]:
# Create the DataFrame for the dataset
date_range = pd.date_range('2017-4-28',periods=8, freq='d')
date_range = date_range.strftime("%Y%m%d")

# Create and open the file, and create the 'Headers' on it
filename = '/users/hnesman/Downloads/DATA_Science/LaVoz.txt'
f = open(filename,'w')
headers = "Date| Title| Encabezado | SecLabel \n"
f.write(headers)

# Iterate over each date on the 'date_range' and access each 'url' for scraping the webpage
for i in date_range:
    url = 'http://www.lavoz.com.ar/ediciones_anteriores?fecha=%s' %i
    
    # Format the date
    date = i
    day = date[6:9]
    month = date[4:6]
    year = date[:4]
    date = (day+'/'+month+'/'+year)
    print date
    
    # Request the url data and parse it using BeautifulSoup
    sauce = uReq(url)
        # does the parsing
    soup = bs.BeautifulSoup(sauce,'html.parser')
       # Gets each (all) the sections for the date 'i' 
    noticias = soup.find_all('div',{'class':'contenido'})
    print len(noticias)
    
    # Loops over each article within 'noticias
    for i in noticias:
        # Gets the title, that includes the Section of the news, and the title. I need to split it. (I could try go get the body of the news) 
        title = i.a['href']
        empty, secLabel, title = title.split('/')
        #I will copy the values of 'title' to 'encabezado' (header) 
        encabezado = title
        print i.a['href'], '\n'
        # I write in the file, the values I took from the webpage
        f.write(date +'|'+ encabezado.replace('-',' ') +'|'+ title.replace('-',' ') + '|' + secLabel + '\n')

f.close()

In [ ]:
# Open the file and put into a dataFrame
LaVoz = pd.read_table('/users/hnesman/Downloads/DATA_Science/LaVoz.txt',delimiter="|")

# Add 'Media' column, which is the label for the name of the Newspaper
LaVoz['Media'] = 'La Voz'

# Set the names of the columns
LaVoz.columns = ['Date','Title','Encabezado','SecLabel','Media']

# Save to .csv file
LaVoz.to_csv('/users/hnesman/Downloads/DATA_Science/LaVoz.csv')
LaVoz.head()

# Los Andes

In [ ]:
# Create and open the file, and set the 'Headers' on the file
filename = '/users/hnesman/Downloads/DATA_Science/LosAndes_.txt'
f = open(filename,'w')
headers = "Date|Title|Encabezado|SecLabel|\n"
f.write(headers)

# Set the period for the analysis
date = '20170428'
periods = 8

# Set the date format I need to use in 'url'
fecha = pd.date_range(date , periods = periods, freq = 'D')
fecha=[x.strftime('%Y-%m-%d').split('-') for x in fecha ]
fecha=['-'.join([x[0],x[1].lstrip('0'),x[2].lstrip('0')]) for x in fecha]

# Access each url ('date'), and scrap the data I need - Using BeautifulSoup for parsing 'html
for i in fecha:
    
    url = 'http://losandes.com.ar/titulos/%s' %i
    sauce = uReq(url)
        # does the parsing
    soup = bs.BeautifulSoup(sauce,'html.parser')
        # gets the tabs containing the date from the BS object

    seccion = soup.findAll('section',{'class':'secciondiario'})
    
    # Define a variable for each section
    economia= seccion[0]
    mundo = seccion[1]
    politica = seccion[3]
    deportes = seccion[4]
    policiales = seccion[5]
    
    # Prepare the format for the variable 'Date'
    date = i
    print date
    year, month, day = date.split('-')

    # I work on the format of the dates (day / month)
    month = '0' + month
    
    if day in ['1','2','3','4','5','6','7','8','9']:
        day = '0' + day
    else:
        day = day
    date = day + '/' + month + '/' + year
    
    # Get all the tags 'li' within the each Section, and from each 'li' I get the tag 'a', and the write on the file
    # cleaning some html code
    not_pol = politica.findAll('li')
    for i in not_pol:
        title = i.a
        f.write(date +'|' + str(title).replace('<a href="/article/','').replace('</a>','').replace('-',' ') + '|'+ '|'+ 'Política' + '\n')

    not_dep = deportes.findAll('li')
    for i in not_dep:
        title = i.a
        f.write(date + '|' + str(title).replace('<a href="/article/','').replace('</a>','').replace('-',' ') +'|'+ '|'+ 'Deportes' + '\n')

    not_econ = economia.findAll('li')
    for i in not_econ:
        title = i.a
        f.write(date + '|' + str(title).replace('<a href="/article/','').replace('</a>','').replace('-',' ') + '|'+ '|'+ 'Economia' + '\n')
    
    not_mundo = mundo.findAll('li')
    for i in not_m:
        title = i.a
        f.write(date + '|' + str(title).replace('<a href="/article/','').replace('</a>','').replace('-',' ') +'|'+ '|'+ 'Mundo' + '\n')
    
    not_polic = policiales.findAll('li')
    for i in noticia:
        title = i.a
        f.write(date + '|' + str(title).replace('<a href="/article/','').replace('</a>','').replace('-',' ') +'|'+ '|'+ 'Policiales' +  '\n')
           
f.close()

In [ ]:
# Open the file and put into a dataFrame
LosAndes = pd.read_table('/users/hnesman/Downloads/DATA_Science/losAndes_.txt',delimiter="|")

# Change names in Headers, and add 'Media' column, which is the label for the name of the Newspaper
LosAndes['Encabezado'] = LosAndes['Title']
LosAndes['Media'] = 'Los Andes'
# I drop a column that was created during the opening of the .txt file #### ATTENTION -  didn't happen in LaVoz, check!
LosAndes = LosAndes.drop('Unnamed: 4', axis =1)
# Set the names of the columns
LosAndes.columns = ['Date','Title','Encabezado','SecLabel','Media']
# Save to .csv file
LosAndes.to_csv('/users/hnesman/Downloads/DATA_Science/LosAndes_.csv',encoding='utf-8')
LosAndes.head()
LosAndes.shape

### Now I have to merge the datasets for each Media, into one.

In [ ]:
# Opening the files into dataFrames
LosAndes = pd.read_csv('/users/hnesman/Downloads/DATA_Science/LosAndes.csv')
LaNacion = pd.read_csv('/users/hnesman/Downloads/DATA_Science/LaNacion.csv')
LaVoz = pd.read_csv('/users/hnesman/Downloads/DATA_Science/LaVoz.csv')

In [ ]:
#Number of Articles per Newspaper
print len(LosAndes)
print len(LaNacion)
print len(LaVoz)


# Do a plot
media_len = [len(LosAndes),len(LaNacion), len(LaVoz)]
Newspaper = [str('LosAndes'),str('LaNacion'), str('LaVoz')]

df = pd.DataFrame({'media_len': media_len, 'Newspaper': Newspaper})
df.hist()

In [ ]:
# Concatenating the dataFrames, drop unused columns and set the name of the 'columns'
MediaNews = pd.concat([LosAndes,LaNacion])
MediaNews = pd.concat([LaVoz,MediaNews])
MediaNews = MediaNews.drop('Unnamed: 0',axis=1)
MediaNews.columns = ['Date', 'Title', 'Encabezado', 'Art_Label', 'Media']
len(MediaNews)

In [ ]:
# Inspection of the dataset and sorting
print type(MediaNews)
print MediaNews.columns
MediaNews = MediaNews.sort_values(by = ['Date','Art_Label'], axis = 0, ascending = True, inplace = False)
MediaNews.head()

In [ ]:
# Create the dictionary for unifying the values of 'Art_Label' and applying the funcion
cmap = {'deportiva': 'deportes', 'Deportes': 'deportes', 'econom\xc3\xada': 'economia', 'economia': 'economia',
        'Economia': 'economia', 'opini\xc3\xb3n': 'editorial', 'opinion': 'editorial',
        'editorial': 'editorial', 'politica': 'politica', 'Pol\xc3\xadtica': 'politica', 'pol\xc3\xadtica': 'politica', 'tecnolog\xc3\xada': 'otros', 'elcampo': 'otros', 'sucesos': 'policiales', 
        'seguridad': 'policiales', 'Policiales': 'policiales', 'mundo': 'elmundo', 'Mundo': 'elmundo', 'elmundo': 'elmundo', 'negocios': 'otros','ciudadanos': 'otros'}

MediaNews['Art_Label'] = MediaNews.Art_Label.apply(lambda x: cmap[str(x)])

In [ ]:
# Save the file to .csv
MediaNews.to_csv('/users/hnesman/Downloads/DATA_Science/MediaNews.csv')

In [ ]:
# Import the libraries for starting the Unsupervised Learning
from gensim.models.word2vec import Word2Vec
from gensim.matutils import Sparse2Corpus
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Open the file into a dataFrame, clean a column "Unnamed: 0' that is created everytime I open the .csv file

data = pd.read_csv('/users/hnesman/Downloads/DATA_Science/MediaNews.csv', sep=',')
data = data.drop('Unnamed: 0', axis=1)
data.columns

In [ ]:
#I want to check how many articles for each Section and Media / Should do a Group By / I have problems with the labels

df2 = data['Art_Label'].value_counts()
df2.plot(kind = 'hist', xlim = (0,400))
print df2

## Now I have to prepare for doing the Word Vector routine

1) I will set up a list of STOP_WORDS in Spanish, I will feed this list after a couple of outcomes of the Vectorization

2) I do the CountVector, I will generate different combinations of 'NGrams', to identify any ambiguous context. (i.e.: papa, is Pope, and Potatoe in Spanish), this risk is increased by the factor of 'casual' writing in Twitter.

3) Last stage, is a visual inspection of the n-NGrams lists created. 

4) I cut a final list of terms, based on my expertise. (Otherwise machines would do everything)

In [ ]:
# I set a list of STOP WORDS in Spanish
# coding: utf8
from __future__ import unicode_literals


STOP_WORDS = set(""" 2004 44 '44 url' 000 100 http https 'http co' 'https co' 'cont co' cont co
none anos ano año años actualmente acuerdo adelante ademas además adrede afirmó agregó ahi ahora ahí
al algo alguna algunas alguno algunos algún alli allí alrededor ambos ampleamos
antano antaño ante anterior antes apenas aproximadamente aquel aquella aquellas
aquello aquellos aqui aquél aquélla aquéllas aquéllos aquí arriba arribaabajo
aseguró asi así atras aun aunque ayer añadió aún
bajo bastante bien breve buen buena buenas bueno buenos
cada casi cerca cierta ciertas cierto ciertos cinco claro comentó como con
conmigo conocer conseguimos conseguir considera consideró consigo consigue
consiguen consigues contigo contra cosas creo cual cuales cualquier cuando
cuanta cuantas cuanto cuantos cuatro cuenta cuál cuáles cuándo cuánta cuántas
cuánto cuántos cómo
da dado dan dar de debajo debe deben debido decir dejó del delante demasiado
demás dentro deprisa desde despacio despues después detras detrás dia dias dice
dicen dicho dieron diferente diferentes dijeron dijo dio donde dos durante día
días dónde
ejemplo el ella ellas ello ellos embargo empleais emplean emplear empleas
empleo en encima encuentra enfrente enseguida entonces entre era eramos eran
eras eres es esa esas ese eso esos esta estaba estaban estado estados estais
estamos estan estar estará estas este esto estos estoy estuvo está están ex
excepto existe existen explicó expresó él ésa ésas ése ésos ésta éstas éste
éstos
fin final fue fuera fueron fui fuimos
general gran grandes gueno
ha haber habia habla hablan habló habrá había habían hace haceis hacemos hacen hacer
hacerlo haces hacia haciendo hago han hasta hay haya he hecho hemos hicieron
hizo horas hoy hubo
igual incluso indicó informo informó intenta intentais intentamos intentan
intentar intentas intento ir
junto
la lado largo las le lejos les llegó lleva llevar lo los luego lugar
mal manera manifestó mas mayor me mediante medio mejor mencionó menos menudo mi
mia mias mientras mio mios mis misma mismas mismo mismos modo momento mucha
muchas mucho muchos muy más mí mía mías mío míos
nada nadie ni ninguna ningunas ninguno ningunos ningún no nos nosotras nosotros
nuestra nuestras nuestro nuestros nueva nuevas nuevo nuevos nunca
ocho os otra otras otro otros
pais para parece parte partir pasada pasado paìs peor pero pesar poca pocas
poco pocos podeis podemos poder podria podriais podriamos podrian podrias podrá
podrán podría podrían poner por porque posible primer primera primero primeros
principalmente pronto propia propias propio propios proximo próximo próximos
pudo pueda puede pueden puedo pues
qeu que quedó queremos quien quienes quiere quiza quizas quizá quizás quién quiénes qué
raras realizado realizar realizó repente respecto rt
sabe sabeis sabemos saben saber sabes salvo se sea sean segun segunda segundo
según seis ser sera será serán sería señaló si sido siempre siendo siete sigue
siguiente sin sino sobre sois sola solamente solas solo solos somos son soy
soyos su supuesto sus suya suyas suyo sé sí sólo
tal tambien también tampoco tan tanto tarde te temprano tendrá tendrán teneis
tenemos tener tenga tengo tenido tenía tercera ti tiempo tiene tienen toda
todas todavia todavía todo todos total trabaja trabajais trabajamos trabajan
trabajar trabajas trabajo tras trata través tres tu tus tuvo tuya tuyas tuyo
tuyos tú
ultimo un una unas uno unos usa usais usamos usan usar usas uso usted ustedes
última últimas último últimos URL url
va vais valor vamos van varias varios vaya veces ver verdad verdadera verdadero
vez vosotras vosotros voy vuelve vuestra vuestras vuestro vuestros
ya yo
""".split())

In [ ]:
# I create my Count Vectorizer in order to Tokenize the articles, I will create a couple CountVect for different
# values of ngram_range(): (1,3) - (2-3) - (3,3). With them I will create dataFrames and inspect which are the
# n-grams that I can use to search on Twitter.

for i in range(1,4):

    cv_i = CountVectorizer(binary = False, stop_words = STOP_WORDS, min_df = 3, ngram_range=(i, 3))

    docs_i = cv_i.fit_transform(data.Encabezado.dropna())
    id2word_i = dict(enumerate(cv_i.get_feature_names()))
    if i == 1:
        NGram_1 = pd.DataFrame({'index' : id2word_i.keys() , 'ngram' : unicode(id2word_i.values()) })
    elif i == 2:
        NGram_2 = pd.DataFrame({'index' : id2word_i.keys() , 'ngram' : unicode(id2word_i.values()) })
    else:
        NGram_3 = pd.DataFrame({'index' : id2word_i.keys() , 'ngram' : unicode(id2word_i.values()) })

In [ ]:
# I copied the result of the CountVectorizer into .csv files so I can inspect by my own which 'tokens' I will
# use for the Twitter Search

NGram_1.to_csv('/users/hnesman/Downloads/DATA_Science/NGram_1.csv')
NGram_2.to_csv('/users/hnesman/Downloads/DATA_Science/NGram_2.csv')
NGram_3.to_csv('/users/hnesman/Downloads/DATA_Science/NGram_3.csv')

# I need techniques to analyze this list with Python, originally I did it with Excel

### Now I have a list of the most relevant terms acording to Media. I will check what people are saying regarding this terms in Twitter.

In [ ]:
import pandas as pd
import numpy as np
import tweepy

try:
    import json
except ImportError:
    import simplejson as json

### I will make use of the Twitter APIs combined with a wrapper called Tweepy

#### To use the Twitter APIs is necessary to get a set of credentials which are provided when you create an app in Twitter. Also the use of the APIs are limited by time and by number of calls, I created two apps, in order to get two set of credentials and speed up the process.

In [ ]:
# I set up my credentials for getting access to Twitter API - App 1
ckey="poglAcoBXr2U41V3BRerO8SoiX"
csecret="uo3MmdHBNQX5O5SV84N70IHkRCIPgOiVKNwIg1PQMznq1ZuvM8j"
atoken="t848659773176385536-XOsIpQcOurrAYHE8qjnq6yq2gtJpw01"
asecret="agIazV001cMvvqtPdfJ1XyksnwSrednnNYQMb1z44PysPb"

In [ ]:
# I define the function 'search_tweets', then to do the query we have to specify a'term' and a 'date'.

#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

# Date as YYYY-MM-DD
def search_tweets(query,date):

	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(ckey, csecret)
	auth.set_access_token(atoken, asecret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.search(q = query,count=10, result_type = 'recent', include_entities =  True)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no more tweets to grab or the list reaches 2000, otherwise you get RateLimitError
	while len(new_tweets) > 0 and len(alltweets) < 2000:
		print "getting tweets before %s" % (oldest)
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.search(q = query ,count=200,max_id=oldest, result_type = 'recent', include_entities = True)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print "...%s tweets downloaded so far" % (len(alltweets))   
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, query, str(tweet.user.name.encode("utf-8")), tweet.created_at, str(tweet.text.encode("utf-8")).replace('\n', ' ').replace('\r', ''), tweet.favorite_count, tweet.retweet_count, tweet.user.followers_count] for tweet in alltweets]
	
	#write the csv	
# 	with open('/users/hnesman/Downloads/DATA_Science/Tweets_Bin/Tweets_Search_%s.csv' % (date) , 'wb') as f:
# 	writer = csv.writer(f,delimiter ='|')
	writer.writerow(["id","query","Username","created_at","text","favorite_count", "retweet_count","user's followers count"])
	writer.writerows(outtweets)
	
	pass

In [ ]:
# Here I perform the search, 'query', 'date' are the arguments. I can put a list of terms in query, but no more than 5
# can take per call, otherwise you get Rate Limited
query = ['bochini']
date = '2017-05-09'

for i in query:
    print i
    f = open('/users/hnesman/Downloads/DATA_Science/Tweets_Bin/%s_Tweets_Search_%s.csv' % (i, date) , 'wb')
    writer = csv.writer(f,delimiter ='|')
    search_tweets(i,date)

In [ ]:
# I set up my credentials for getting access to Twitter API
ckey="btyxpVnvWkPG1pmnwPLByzse4"
csecret="BxvPvVzpW7HwPwy0tWW4skOwcGfFEZ01CtRVPwSpoyjXUVUVf8"
atoken="848659773176385536-KtFLOtwUxxGl20cpcyIG5aFusEaR7iT"
asecret="pCEy2mgprgfXEB2o76msPV3uFTmfbvGFRozYbCFj6TcFc"

# Initialise Tweepy
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth)

In [ ]:
# I define 'search_tweets'

#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

# Date as YYYY-MM-DD
def search_tweets(query,date):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(ckey, csecret)
	auth.set_access_token(atoken, asecret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.search(q = query,count=10, result_type = 'recent', include_entities =  True)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0 and len(alltweets) < 2000:
		print "getting tweets before %s" % (oldest)
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.search(q = query ,count=200,max_id=oldest, result_type = 'recent', include_entities = True)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print "...%s tweets downloaded so far" % (len(alltweets))   
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, query, str(tweet.user.name.encode("utf-8")), tweet.created_at, str(tweet.text.encode("utf-8")).replace('\n', ' ').replace('\r', ''), tweet.favorite_count, tweet.retweet_count, tweet.user.followers_count] for tweet in alltweets]
	
	#write the csv	
# 	with open('/users/hnesman/Downloads/DATA_Science/Tweets_Bin/Tweets_Search_%s.csv' % (date) , 'wb') as f:
# 	writer = csv.writer(f,delimiter ='|')
	writer.writerow(["id","query","Username","created_at","text","favorite_count", "retweet_count","user's followers count"])
	writer.writerows(outtweets)
	
	pass

In [ ]:
query = ['jefe policía ciudad', 'video analista político']
date = '2017-05-06'

for i in query:
    print i
    f = open('/users/hnesman/Downloads/DATA_Science/Tweets_Bin/%s_Tweets_Search_%s.csv' % (i, date) , 'wb')
    writer = csv.writer(f,delimiter ='|')
    search_tweets(i,date)

#### I saved all the results from the calls in .csv files in a specific folder. The next step is to put together all the files in one dataset.

In [ ]:
#!/usr/bin/env python
# Este archivo usa el encoding: utf-8
import unicodedata

In [ ]:
from os import listdir
from os.path import isfile, join

## Set up all the search Tweets into one file:

1) Concatenate the files into a pd.DataFrame

2) Save it into a .csv file

In [ ]:
# Create a list with the names of all the files where I have all the Twitter Search Files
mypath = '/users/hnesman/Downloads/DATA_Science/Tweets_Bin'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
type(onlyfiles)

In [ ]:
# I delete a fake name that is generated by the loop above
del onlyfiles[0]

print onlyfiles

In [ ]:
cd /users/hnesman/Downloads/DATA_Science/Tweets_Bin

# Defines a 'for-loop' for sequencial reading and concatenating all the files in the folder
df = pd.concat((pd.read_csv(i,delimiter = '|') for i in onlyfiles))

# I drop a column that is created when I concatenated the dataframes
df = df.drop(["id;query;Username;created_at;text;favorite_count;retweet_count;user's followers count"], axis = 1)

len(df)

In [ ]:
# Identify which tweets are retweets

df['Is RT?'] = df['text'].map(lambda x: 1 if 'RT' in str(x) else 0)

In [ ]:
# I clean the url's

df['text'] = df['text'].replace('((www\.[^\s]+)|(https?://[^\s]+))','', regex=True)

In [ ]:
# I clean the RT's from the text

df['text'] = df['text'].replace('RT','', regex=True)

In [ ]:
# Check the amount of Tweets in my dataset
len(df)

In [ ]:
# I delete the duplicate tweets

df_proc = df.drop_duplicates('id')
print len(df_proc)

In [ ]:
# I drop all the rows with any 'Nan' values and reindex the values

df_proc = df_proc.dropna(how='any')
df_proc.reset_index(inplace=True)
print len(df_proc)

In [ ]:
# Drop a column 'index' created
df_proc = df_proc.drop("index", axis = 1)

I noticed that the results of the Search from Twitter includes tweets not only in Spanish, there are a few tweets in different languages (i.e.: Italian, Portuguese, Japanese!)

One of the API Search parameters is 'Lang' which allows select the language of the , but I didn't bother because even in Twitter's Dev site they recommend to use a Language detector.

### I will use Langid and Langdetect for filtering only tweets in Spanish

In [ ]:
#!/usr/bin/env python
# Este archivo usa el encoding: utf-8
import langid  
from langdetect import detect  
# import textblob

def langid_safe(tweet):  
    try:
        return langid.classify(tweet)[0]
    except Exception as e:
        pass

def langdetect_safe(tweet):  
    try:
        return detect(tweet)
    except Exception as e:
        pass

df_proc['lang_langid'] = df_proc.text.apply(langid_safe)  
df_proc['lang_langdetect'] = df_proc.text.apply(langdetect_safe)  

# tweets.to_csv('tweets_parsed2.csv', encoding='utf-8')

df_proc = df_proc.query("lang_langdetect == 'es' or lang_langid == 'es' ")

#### Now I have narrowed my dataset to Spanish Tweets, the following step is to filter out the Retweets as they don't provide further information

In [ ]:
# I drop the retweets

df_clean = df_proc[df_proc['Is RT?']==0]

# And reset the index to keep tidy
df_clean.reset_index(inplace=True)
len(df_clean)

I got 8663 tweets left

In [ ]:
# I delete the column 'Index' created when I do 'reset_index'
df_clean = df_clean.drop("index", axis = 1)
df_clean.tail()

In [ ]:
# I save my file into .csv, for then being fit it into the Model
df_clean.to_csv('/users/hnesman/Downloads/DATA_Science/Argentina_TWEETS.csv', index=True)

### Now I will label the tweets, the values I use are P: positive, N: negative, Neu: neutral/no clear sentiment

I used Excel for this task, I manually labeled the tweets, saved it in a different file, because I had troubles opening the Tweets File with the 'polarity' column. 

In [ ]:
# I open the file with the polarity of the tweets

polarity = pd.read_csv('/users/hnesman/Downloads/DATA_Science/Index_polarity.csv', sep = ';',header = None)

Note: Misteriously a couple of tweets were missed during the export from dataframe to csv. It's something to bear in mind when exports to Excel are involved

In [ ]:
# Setting up the name of the columns in the Polarity dataset 

polarity.columns = ['index','cPolarity']
polarity.head()

In [ ]:
# Merging both datasets (there has to be a column in common)

df_labels = pd.merge(df_clean, polarity, on='index')

In [ ]:
# Set up the names of the columns of the new dataframe

df_labels.columns = ['index','Username','created_at','favorite_count','id','q','retweet_count','text','user followers count'
                    ,'is RT?', 'lang_langid','lang_langdetect','cPolarity']
df_labels.tail()

### Now I have my Tweets dataset with polarity, I will clean some information that is irrelevant (to my understanding), and will map a category to each Tweet

1) Delete all the Tweets with 'query' = 'sugita' (it is an irrelevant result that came from the search on Media, I shouldn't have considered initially but I didn't know at the time I put together the dataset)

2) I will map a category to each query, this might be usefull for Visualisation of the Arg Tweets Dataset.

3) I will drop the tweets with 'Polarity' = ['Nan','Neu']

In [ ]:
# Delete Tweets with query = 'sugita'

tweets_arg = df_labels[df_labels.q != 'sugita']

In [ ]:
# Map categories for the 'Search Terms' and create a feature with its value

cmap = {'alicia kirchner' : 'politics','vaticano' : 'religion/politics','2x1' : 'politics','macri' : 'politics','fernando cartasegna' : 'politics','afa' : 'sports/politics','macri trump' : 'politics/world','delitos lesa humanidad' : 'politics','"corte suprema"' : 'politics','CFK' : 'politics','"dt de boca"' : 'sports','grupo macri' : 'politics','planes sociales' : 'politics','maqueda' : 'politics','francisco' : 'religion/politics','lilian tintori' : 'politics/world','Barros Schelotto' : 'sports','river' : 'sports','"plaza de mayo"' : 'politics','kirchnerismo' : 'politics'}
tweets_arg['category'] = tweets_arg.q.map(lambda x: cmap[str(x)])

In [ ]:
# Map the values of Polarity, in order to filter and manipulate the Tweets based in this feature

cmap = {'P': 1,'nan': 2,'Neu': 2, 'N': 0, '0':0, 'NEu': 2,}
tweets_arg['Polarity'] = tweets_arg.cPolarity.map(lambda x: cmap[str(x)])
tweets_arg.head()

In [ ]:
# I keep the tweets with Polarity either Positive or Negative

tweets_arg_2 = tweets_arg[tweets_arg.Polarity != 2]

## Exploring the Twitter Dataset

### I will narrow my analysis to the tweets that have been labeled with a positive/negative sentiment


#### As the plot provides a measure of positivity ratio: Positive tweets/ Total Tweets, sorting by Polarity for each 'q' term we get the ratio

In [ ]:
import seaborn as sb

In [ ]:
%matplotlib inline

In [ ]:
# First I get an overview of which are the sentiments regarding "Category"

p = sb.factorplot(x='category', 
              y='Polarity',
              kind='bar',
              data = tweets_arg_2,
              size = 5,
            )
p.set_xticklabels(rotation=45, horizontalalignment='right')

#### I will take it to a more granular level using "q"

In [ ]:
p = sb.factorplot(x='q', 
              y='Polarity',
              kind='bar',
              data = tweets_arg_2,
              size = 6
            )
p.set_xticklabels(rotation=45, horizontalalignment='right')

In [ ]:
## I want to see what are the terms that generate the most positive and most negative feelings

- I will sort from higher to lower, to see the most positive

- I will sort from lower to higher, to see the most negative

In [ ]:
# I create the dataframes for the most positive terms

# I get the top ten results for 'q'
query_most = tweets_arg_2.groupby('q').sum().sort_values(by='Polarity', ascending= False)
query_top_ten = query_most.iloc[:10]
(query_top_ten)

In [ ]:
# I do the plot for the Top Ten

p = sb.factorplot(x='q', 
              y='Polarity',
              kind='bar',
              data = query_top_ten,
              size = 6
            )
p.set_xticklabels(rotation=45, horizontalalignment='right')

In [ ]:
# I get the last ten results for 'q'
query_less = tweets_arg_2.groupby('q').sum().sort_values(by='Polarity', ascending= True)
query_last_ten = query_less.iloc[:10]
(query_last_ten)

In [ ]:
# I do the plot for the Last Ten


p = sb.factorplot(x='q', 
              y='Polarity',
              kind='bar',
              data = query_last_ten,
              size = 6
            )
p.set_xticklabels(rotation=45, horizontalalignment='right')

### I want to know which are the top 5 tweets in terms of popularity (measured by Favorite's Count)

In [ ]:
# I sort the dataframe by 'Favorite Count' and keep the top 5

favorite_tweets = tweets_arg_2.sort_values(by='favorite_count', ascending =  False).iloc[:5][['Username','user followers count','favorite_count','text']]
favorite_tweets.to_csv('/users/hnesman/downloads/DATA_Science/favoritetweets.csv')
favorite_tweets

# Wordcloud

### To get another type of glance to the data we can use this type of charts, I will split the dataframe accordingly to its positive or negative feelings

1) Split the dataset between positive and negative

2) perform Wordclouds for each

In [ ]:
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# I separate the positive and negative Tweets

positive = tweets_arg_2[tweets_arg_2.Polarity == 1]
print len(positive)
negative = tweets_arg_2[tweets_arg_2.Polarity == 0]
print len(negative)

In [ ]:
# I will replace a few words in the Datasets that don't apport any information ('que', 'la','lo','de','el')

positive['text'] = positive['text'].replace('que | la | el | lo | de','', regex=True)       
negative['text'] = negative['text'].replace('que | la | el | lo | de','', regex=True)         

In [ ]:
# Positive Wordcloud
wordcloud = WordCloud(width = 1000, height = 500).generate(' '.join(positive1['text']))

plt.figure(figsize=(15,8))

plt.imshow(wordcloud)

plt.axis("off")

plt.show()

In [ ]:
# Negative Wordcloud
wordcloud = WordCloud(width = 1000, height = 500).generate(' '.join(negative['text']))

plt.figure(figsize=(15,8))

plt.imshow(wordcloud)

plt.axis("off")

plt.show()

In [ ]:
# I save my file into .csv, for then being fit it into the Model
# df_clean.to_csv('/users/hnesman/Downloads/DATA_Science/Argentina_TWEETS_cuidado.csv', index=True)

# Now the last part of the project, perform the Sentiment Analysis

### To find the model I will use the TASS dataset, created by the SEPLN (Sociedad Española para el Procesamiento del Lenguaje Natural)

Is composed by 3 files of labelled Tweets in Spanish


### Open the .xml files, get the data I need, save it into a tuple, then save it into a .csv

In [ ]:
# I open the file 'General Tweet Train Tagged.csv'
try:
    general_tweets_corpus_train = pd.read_csv('/users/hnesman/Downloads/DATA_Science/TASS/general-tweets-train-tagged.csv', encoding='utf-8')
except:

    from lxml import objectify
    xml = objectify.parse(open('/users/hnesman/Downloads/DATA_Science/TASS/general-tweets-train-tagged.xml'))
    #sample tweet object
    root = xml.getroot()
    general_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity', 'agreement'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiments.polarity.value.text, tweet.sentiments.polarity.type.text]))
        row_s = pd.Series(row)
        row_s.name = i
        general_tweets_corpus_train = general_tweets_corpus_train.append(row_s)
    general_tweets_corpus_train.to_csv('/users/hnesman/Downloads/DATA_Science/TASS_Parsed/general-tweets-train-tagged.csv', index=False, encoding='utf-8')

len(general_tweets_corpus_train)

In [ ]:
# I open the file 'Stompol Tweets Train Tagged'
try:
    stompol_tweets_corpus_train = pd.read_csv('/users/hnesman/Downloads/DATA_Science/TASS/stompol-tweets-train-tagged.csv', encoding='utf-8')
except:

    from lxml import objectify
    xml = objectify.parse(open('/users/hnesman/Downloads/DATA_Science/TASS/stompol-tweets-train-tagged.xml'))
    #sample tweet object
    root = xml.getroot()
    stompol_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [' '.join(list(tweet.itertext())), tweet.sentiment.get('polarity')]))
        row_s = pd.Series(row)
        row_s.name = i
        stompol_tweets_corpus_train = stompol_tweets_corpus_train.append(row_s)
    stompol_tweets_corpus_train.to_csv('/users/hnesman/Downloads/DATA_Science/TASS_Parsed/stompol-tweets-train-tagged.csv', index=False, encoding='utf-8')

In [ ]:
len(stompol_tweets_corpus_train)

In [ ]:
# I open the file 'Politics 2013 Tweets Test Tagged'
try:
    general_tweets_corpus_train = pd.read_csv('/users/hnesman/Downloads/DATA_Science/TASS/politics2013-tweets-test-tagged.csv', encoding='utf-8')
except:

    from lxml import objectify
    xml = objectify.parse(open('/users/hnesman/Downloads/DATA_Science/TASS/politics2013-tweets-test-tagged.xml'))
    #sample tweet object
    root = xml.getroot()
    pol_2013_tweets_corpus_train= pd.DataFrame(columns=('content', 'polarity', 'agreement'))
    tweets = root.getchildren()
    for i in range(0,len(tweets)):
        tweet = tweets[i]
        row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiments.polarity.value.text, tweet.sentiments.polarity.type.text]))
        row_s = pd.Series(row)
        row_s.name = i
        pol_2013_tweets_corpus_train = pol_2013_tweets_corpus_train.append(row_s)
    pol_2013_tweets_corpus_train.to_csv('/users/hnesman/Downloads/DATA_Science/TASS_Parsed/politics2013-tweets-test-tagged.csv', index=False, encoding='utf-8')

In [ ]:
len(pol_2013_tweets_corpus_train)

In [ ]:
# I concatenate the 3 dataFrames into the dataframe 'tweets_corpus'

tweets_corpus = pd.concat([
        pol_2013_tweets_corpus_train,
        stompol_tweets_corpus_train,
        general_tweets_corpus_train
        
    ])
len(tweets_corpus)

# Building the Model

## Set up the labels of the variable 'Polarity', as I want to predict binary values, I have to transform some values of this variable

    1) Drop all the rows with polarity value = 'NEU' and 'NONE'
    2) Join N and N+, into a new value = 0
    3) Join P and P+, into a new value = 1

In [ ]:
# I do an inspection of the values in the variable 'polarity'

print tweets_corpus.groupby('polarity').count()
tweets = tweets_corpus.copy()
print tweets.columns
len(tweets)

In [ ]:
# Create a cmap for the values of 'polarity'

cmap = {'P+': 1, 'P': 1,'NONE': 2,'NEU': 2, 'N': 0, 'N+': 0}
tweets['cpolarity'] = tweets.polarity.apply(lambda x: cmap[str(x)])
tweets.head()

In [ ]:
# Check everything run alright
print tweets.groupby('cpolarity').count()

In [ ]:
# Drop all the tweets with 'cpolarity' = 2 (accordingly to the map are the 'None' or 'Neu' values)
data = tweets[tweets.cpolarity != 2]
print data.groupby('cpolarity').count()

In [ ]:
data.to_csv('/users/hnesman/downloads/DATA_Science/TASS.csv', index = False, encoding = 'utf-8')

In [ ]:
# The lenght of the set and the distribution of values: 1 - Positive, 0 - Negative
print len(data)
data.hist()

## Now my dataset is ready to train a model for Classification:

# Find a model capable to predict Sentiment Analysis on TASS dataset.

Models I will try:

1) Random Forest Classifier

2) Logistic Regression

Every Model is built under the frame Grid Search / Cross Validation, to assure the best results:

The input of the models are the Word Vectors, and there are a few different ways to approach them:

- CountVectorizer (Binary: False/True) / (Freq / Presence)
- Tf-idF (I could extract the features ('n-grams') and compare both results

I will use both Vectorizing approaches across both models, with the option that provides me best results I will address my dataset (Tweets from Argentina)

## STOP WORDS list

I will use the one defined on the Media Research Section

In [ ]:
# I import some libraries I will need

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search, cross_validation
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import feature_extraction, ensemble, cross_validation, metrics

#### I will use CountVectorizer to Tokenise, filter the 'STOP WORDS' and create the Word Vector:
Note:

1) The parameter 'max_features' is set to 100.

2) The method 'get_feature_names' is useful to detect words that don't add much information, and so, is better to include them in the STOP_WORDS list. I iterate this task until I get satisfactoy results.

3) Check that if the parameter 'binary':
    a) False, the vector gets the Frequency of the n-gram

    b) True, the vector gets the Presence (yes/no) of the n-gram

### I define a function for visualising the performance of the best estimator

In [ ]:
## Here's a function that plots the ROC curve as well as report model accuracy

def plot_bi_roc (X_transformed, y, model, title_text):
    y_hat = model.predict(X_transformed)
    fpr, tpr, thresholds = metrics.roc_curve(y, y_hat)
    
    plt.figure()
    plt.plot(fpr, tpr, label = 'ROC curve (area = %0.2f)' % metrics.auc(fpr, tpr))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([.0, 1.])
    plt.ylim([.0, 1.1])
    plt.xlabel('FPR/Fall-out')
    plt.ylabel('TPR/Sensitivity')
    plt.title(title_text+' Sentiment ROC')
    plt.legend(loc = 'lower right')
    plt.show()
    
#END plot_bi_roc ()

### I define a function for visualising the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
import itertools
# This function plots a confusion matrix 
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    return

# Cvectorizer_presen

In [ ]:
# CountVectorizer with Binary = True

content = data['content']

Cvectorizer_presen = CountVectorizer(max_features = 50, 
                             ngram_range=(1, 2), 
                             stop_words= STOP_WORDS,
                             binary=True)

# Use `fit` to learn the vocabulary of the titles
Cvectorizer_presen.fit(content)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
# X_presen = Cvectorizer_presen.transform(content)

X_presen = Cvectorizer_presen.transform(content).toarray()
y_presen = data['cpolarity']
presen = pd.Series(Cvectorizer_presen.get_feature_names())

## I split the data set in Test and Train

In [ ]:
X_presen_train,X_presen_test,y_presen_train,y_presen_test=train_test_split(X_presen,y_presen,test_size=0.3)

print len(X_presen_train)
print len(X_presen_test)

# Random Forest Classifier

Cross Validation enhances the learning capabilities of the model. Cross Validation generates several models in cross sections of the data, measures the performance of each and calculates the mean performance. In this way, we are preventing from overfitting the model.

Grid Search optimizes the parameters of the model, providing the best estimator parameters and its scores.

In [ ]:
model = RandomForestClassifier(n_estimators = 5)


gs_RFC_presen = grid_search.GridSearchCV(estimator = model,
                             param_grid = {'n_estimators': [i for i in range(1,52,10)],
                                          "max_depth": [3, 5],
                                          "bootstrap": [True, False],
                                           "class_weight": [None,'balanced'],
                                          "criterion": ["gini"]},
                             cv = cross_validation.KFold(n=len(X_presen_train), n_folds=5), scoring='roc_auc')

gs_RFC_presen.fit(X_presen_train, y_presen_train)
print (gs_RFC_presen.best_estimator_)
print (gs_RFC_presen.best_score_)

best_RFC_model_presen = gs_RFC_presen.best_estimator_

In [ ]:
The score is over 0.5, so we can say that the model is acceptable.

### Testing the model - ROC curve 

In [ ]:
plot_bi_roc(X_presen_test, y_presen_test, best_RFC_model_presen, 'RFC')

### Confusion Matrix

In [ ]:
y_pred=gs_RFC_presen.predict(X_presen_test)
plot_confusion_matrix(confusion_matrix(y_presen_test,y_pred),y_presen_test.unique())

# Logistic Regression

- I will use the L2 penalisation as L1 is recommended when we have more features than observations. In this case we have 100 features and 7001 observations (tweets)
- I will add `random_state` in `param_grid` for Grid Search.

In [ ]:
gs_LR_presen = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced'],
               'random_state':[i for i in range(100,150,10)]},
    cv=cross_validation.KFold(n=len(X_presen_train), n_folds=10),
    scoring='roc_auc'
)

gs_LR_presen.fit(X_presen_train, y_presen_train)
print (gs_LR_presen.best_estimator_)
print (gs_LR_presen.best_score_)

best_LR_model_presen = gs_LR_presen.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
# We do the visualisation of the ROC Curve

# plot_bi_roc(X_presen_test, y_presen_test, gs_LR_presen, 'Reg Log')
plot_bi_roc(X_presen_test, y_presen_test, best_LR_model_presen, 'Reg Log')

### Confusion Matrix

In [ ]:
y_pred=gs_LR_presen.predict(X_presen_test)
plot_confusion_matrix(confusion_matrix(y_presen_test,y_pred),y_presen_test.unique())

# CVectorizer_freq

In [ ]:
# CountVectorizer with Binary = False

content = data['content']

Cvectorizer_freq = CountVectorizer(max_features = 50, 
                             ngram_range=(1, 2), 
                             stop_words= STOP_WORDS,
                             binary=False)

# Use `fit` to learn the vocabulary of the titles
Cvectorizer_freq.fit(content)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
# X_freq = Cvectorizer_freq.transform(content)

X_freq = Cvectorizer_freq.transform(content).toarray()
y_freq = data['cpolarity']

freq = pd.Series(Cvectorizer_freq.get_feature_names())

### Split the data

In [ ]:
X_freq_train,X_freq_test,y_freq_train,y_freq_test=train_test_split(X_freq,y_freq,test_size=0.3)

print len(X_freq_train)
print len(X_freq_test)

## Random Forest Classifier

In [ ]:
model = RandomForestClassifier(n_estimators = 5)


gs_RFC_freq = grid_search.GridSearchCV(estimator = model,
                             param_grid = {'n_estimators': [i for i in range(1,52,10)],
                                          "max_depth": [3, 5],
                                          "bootstrap": [True, False],
                                           "class_weight": [None,'balanced'],
                                          "criterion": ["gini"]},
                             cv = cross_validation.KFold(n=len(X_presen_train), n_folds=5), scoring='roc_auc')

gs_RFC_freq.fit(X_freq_train, y_freq_train)
print (gs_RFC_freq.best_estimator_)
print (gs_RFC_freq.best_score_)

best_RFC_model_freq = gs_RFC_freq.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
plot_bi_roc(X_freq_test, y_freq_test, best_RFC_model_freq, 'RFC')

### Confusion Matrix

In [ ]:
y_pred=gs_RFC_freq.predict(X_freq_test)
plot_confusion_matrix(confusion_matrix(y_freq_test,y_pred),y_freq_test.unique())

## Logistic Regression

- I will use the L2 penalisation as L1 is recommended when we have more features than observations. In this case we have 100 features and 7001 observations (tweets)
- I will add `random_state` in `param_grid` for Grid Search.

In [ ]:
gs_LR_freq = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced'],
               'random_state':[i for i in range(100,150,10)]},
    cv=cross_validation.KFold(n=len(X_presen_train), n_folds=10),
    scoring='roc_auc'
)

gs_LR_freq.fit(X_freq_train, y_freq_train)
print (gs_LR_freq.best_estimator_)
print (gs_LR_freq.best_score_)

best_LR_model_freq = gs_LR_freq.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
plot_bi_roc(X_freq_test, y_freq_test, best_LR_model_freq, 'Log Reg')

### Confusion Matrix

In [ ]:
y_pred=gs_LR_freq.predict(X_freq_test)
plot_confusion_matrix(confusion_matrix(y_freq_test,y_pred),y_freq_test.unique())

# Tf-idF

In [ ]:
# The Tf-idF option for Word Vectorization

content = data['content']

Tfvectorizer = TfidfVectorizer(max_df=0.5, max_features = 50, ngram_range = (1,2),
                                 stop_words=STOP_WORDS)
# X_TfidF = Tfvectorizer.fit_transform(content)

X_TfidF = Tfvectorizer.fit_transform(content).toarray()
TfidF = pd.Series(Tfvectorizer.get_feature_names())

y_TfidF = data['cpolarity']

# X.shape

### Split the data

In [ ]:
X_TfidF_train,X_TfidF_test,y_TfidF_train,y_TfidF_test=train_test_split(X_TfidF,y_TfidF,test_size=0.3)

print len(X_TfidF_train)
print len(X_TfidF_test)

## Random Forest Classifier

In [ ]:
model = RandomForestClassifier(n_estimators = 5)


gs_RFC_TfidF = grid_search.GridSearchCV(estimator = model,
                             param_grid = {'n_estimators': [i for i in range(1,52,10)],
                                          "max_depth": [3, 5],
                                          "bootstrap": [True, False],
                                           "class_weight": [None,'balanced'],
                                          "criterion": ["gini"]},
                             cv = cross_validation.KFold(n=len(X_presen_train), n_folds=5), scoring='roc_auc')

gs_RFC_TfidF.fit(X_presen_train, y_presen_train)
print (gs_RFC_TfidF.best_estimator_)
print (gs_RFC_TfidF.best_score_)

best_RFC_model_TfidF = gs_RFC_TfidF.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
plot_bi_roc(X_TfidF_test, y_TfidF_test, best_RFC_model_TfidF, 'RFC')

### Confusion Matrix

In [ ]:
y_pred=gs_RFC_TfidF.predict(X_TfidF_test)
plot_confusion_matrix(confusion_matrix(y_TfidF_test,y_pred),y_TfidF_test.unique())

## Logistic Regression

- I will use the L2 penalisation as L1 is recommended when we have more features than observations. In this case we have 100 features and 7001 observations (tweets)
- I will add `random_state` in `param_grid` for Grid Search.

In [ ]:
gs_LR_TfidF = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced'],
               'random_state':[i for i in range(100,150,10)]},
    cv=cross_validation.KFold(n=len(X_presen_train), n_folds=10),
    scoring='roc_auc'
)

gs_LR_TfidF.fit(X_TfidF_train, y_TfidF_train)
print (gs_LR_TfidF.best_estimator_)
print (gs_LR_TfidF.best_score_)

best_LR_model_TfidF = gs_LR_TfidF.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
plot_bi_roc(X_TfidF_test, y_TfidF_test, best_LR_model_TfidF, 'Log Reg')

### Confusion Matrix

In [ ]:
y_pred=gs_LR_TfidF.predict(X_TfidF_test)
plot_confusion_matrix(confusion_matrix(y_TfidF_test,y_pred),y_TfidF_test.unique())

# Now I will perform the Sentiment Analysis over the dataset I collected

Log Reg models perform better, so I am choose this one for my dataset. I will try Presen and TfidF for vectorization

In [ ]:
# I open the file and check that is the correct one
#!/usr/bin/env python
# Este archivo usa el encoding: utf-8
df_label = pd.read_csv('/users/hnesman/Downloads/DATA_Science/Tweets_Arg_Label.csv')

In [ ]:
# Drop the column 'Unnamed: 0', that is generated when opening the file

df_label = df_label.drop(["Unnamed: 0","index"], axis = 1)

## Count Vectorizer Presence / (Binary = True)  

In [ ]:
# Check there is no null values
df_label.text.isnull().describe()

In [ ]:
#!/usr/bin/env python
# Este archivo usa el encoding: utf-8

# content = df_label[('text')].dropna(inplace=True)

Cvect_presen_TA = CountVectorizer(max_features = 50, 
                             ngram_range=(1, 2), 
                             stop_words= STOP_WORDS,
                             binary=True)

X_presen_TA = Cvect_presen_TA.fit_transform(df_label.text.str.decode('utf-8', 'ignore')).toarray()
y_presen_TA = df_label['Polarity']


# TO GET A SERIES WITH THE FEATURES FROM CVECT_PRESEN
presen_TA = pd.Series(Cvect_presen_TA.get_feature_names())


### Split the data (Train, Test)

In [ ]:
X_presen_TA_train,X_presen_TA_test,y_presen_TA_train,y_presen_TA_test=train_test_split(X_presen_TA,y_presen_TA,test_size=0.3)

print len(X_presen_TA_train)
print len(X_presen_TA_test)

## Logistic Regression

In [ ]:
gs_LR_presen_TA = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced'],
               'random_state':[i for i in range(100,150,10)]},
    cv=cross_validation.KFold(n=len(X_presen_TA_train), n_folds=10),
    scoring='roc_auc'
)

gs_LR_presen_TA.fit(X_presen_TA_train, y_presen_TA_train)
print (gs_LR_presen_TA.best_estimator_)
print (gs_LR_presen_TA.best_score_)

best_LR_model_presen_TA = gs_LR_presen_TA.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
plot_bi_roc(X_presen_TA_test, y_presen_TA_test, best_LR_model_presen_TA, 'Log Reg')

### Confusion Matrix

In [ ]:
y_pred=gs_LR_presen_TA.predict(X_presen_TA_test)
plot_confusion_matrix(confusion_matrix(y_presen_TA_test,y_pred),y_presen_TA_test.unique())

## TfidF Vectorization

In [ ]:
# The Tf-idF option for Word Vectorization

# content = data['content']

TfidF_TA_vectorizer = TfidfVectorizer(max_df=0.5, max_features = 50, ngram_range = (1,2),
                                 stop_words=STOP_WORDS)

# X_TfidF = Tfvectorizer.fit_transform(content)

# TO CREATE THE WORD VECTOR - TFIDF
X_TfidF_TA = TfidF_TA_vectorizer.fit_transform(df_label.text.str.decode('utf-8', 'ignore')).toarray()
y_TfidF_TA = df_label['Polarity']

# TO GET A SERIES WITH THE FEATURES FROM TFIDF
TfidF_TA = pd.Series(TfidF_TA_vectorizer.get_feature_names())

### Split the Data

In [ ]:
X_TfidF_TA_train,X_TfidF_TA_test,y_TfidF_TA_train,y_TfidF_TA_test=train_test_split(X_TfidF_TA,y_TfidF_TA,test_size=0.3)

print len(X_TfidF_TA_train)
print len(X_TfidF_TA_test)

## Logistic Regression

In [ ]:
gs_LR_TfidF_TA = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced'],
               'random_state':[i for i in range(100,150,10)]},
    cv=cross_validation.KFold(n=len(X_TfidF_TA_train), n_folds=10),
    scoring='roc_auc'
)

gs_LR_TfidF_TA.fit(X_TfidF_TA_train, y_TfidF_TA_train)

print (gs_LR_TfidF_TA.best_estimator_)
print (gs_LR_TfidF_TA.best_score_)

best_LR_model_TfidF_TA = gs_LR_TfidF_TA.best_estimator_

### ROC curve the best estimator´s performance

In [ ]:
plot_bi_roc(X_TfidF_TA_test, y_TfidF_TA_test, best_LR_model_TfidF_TA, 'Log Reg')

### Confusion Matrix

In [ ]:
y_pred=gs_LR_TfidF.predict(X_TfidF_test)
plot_confusion_matrix(confusion_matrix(y_TfidF_test,y_pred),y_TfidF_test.unique())